This notebook calculates the change in direct damage to specified assets

In [4]:
# Imports
import configparser
from pathlib import Path
import pathlib
from direct_damages import damagescanner_rail_track as ds
import pandas as pd
import geopandas as gpd
# import datetime
from ci_adapt_utilities import *
import pickle

In [5]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'

In [6]:
# Read exposure data (OSM, OpenStreetMap contributors (2024) / osm-flex)
assets_path = data_path / asset_data
assets = gpd.read_file(assets_path)
assets = gpd.GeoDataFrame(assets).set_crs(4326).to_crs(3857)
assets = assets.loc[assets.geometry.geom_type == 'LineString']
assets = assets.rename(columns={'railway' : 'asset'})

# # Drop bridges and tunnels
# assets = assets.loc[~(assets['bridge'].isin(['yes']))]
# assets = assets.loc[~(assets['tunnel'].isin(['yes']))]

assets = assets[assets['railway:traffic_mode']!='"passenger"']
assets = assets[assets['asset']=='rail']

assets = assets.reset_index(drop=True)

# Add buffer to assets to do area intersect and create dictionaries for quicker lookup
buffered_assets = ds.buffer_assets(assets)
geom_dict = assets['geometry'].to_dict()
type_dict = assets['asset'].to_dict()

print(f"{len(assets)} assets loaded.")

# Read vulnerability and maximum damage data from Nirandjan, S., et al. (2024)
curve_types = {'rail': ['F8.1']}
infra_curves, maxdams = ds.read_vul_maxdam(data_path, hazard_type, infra_type)
max_damage_tables = pd.read_excel(data_path / vulnerability_data / 'Table_D3_Costs_V1.0.0.xlsx',sheet_name='Cost_Database',index_col=[0])
print(f'Found matching infrastructure curves for: {infra_type}')

2822 assets loaded.
Found matching infrastructure curves for: rail


In [7]:
# open pickled hazard-asset overlay and hazard intensity data
with open(interim_data_path / 'overlay_assets_flood_DERP_RW_L_4326_2080411370.pkl', 'rb') as f:
    overlay_assets = pickle.load(f)
with open(interim_data_path / 'hazard_numpified_flood_DERP_RW_L_4326_2080411370.pkl', 'rb') as f:
    hazard_numpified_list = pickle.load(f)


In [8]:

# Set up the path
adapt_path = Path(r'C:\Data\interim\adaptations\test_haz_level_adapt.geojson')

# Load the polygon of the protected area
protected_area = gpd.read_file(adapt_path).to_crs(3857)

# Filter assets that are within the protected area
filtered_assets = gpd.overlay(assets, protected_area, how='intersection')
f_assets = assets.loc[(assets['osm_id'].isin(filtered_assets['osm_id']))]
f_assets.head(3)
# assets = assets.loc[~(assets['bridge'].isin(['yes']))]


,osm_id,asset,name,gauge,electrified,voltage,bridge,maxspeed,service,tunnel,other_tags,railway:traffic_mode,usage,geometry,buffered
12,7990468,rail,None,1435,contact_line,15000,None,120,None,None,"""frequency""=>""16.7"",""operator""=>""DB Netz AG"",""...","""mixed""","""main""","LINESTRING (810131.824 6545141.628, 810131.034...","POLYGON ((810131.0332578894 6545143.573510949,..."
269,33506714,rail,Rechte Rheinstrecke,1435,contact_line,15000,yes,100,None,None,"""frequency""=>""16.7"",""layer""=>""1"",""operator""=>""...","""mixed""","""main""","LINESTRING (806342.732 6547553.449, 806364.083...","POLYGON ((806364.0829647976 6547544.350868201,..."
270,33506715,rail,Rechte Rheinstrecke,1435,contact_line,15000,None,100,None,None,"""embankment""=>""yes"",""frequency""=>""16.7"",""opera...","""mixed""","""main""","LINESTRING (806681.210 6547474.401, 806676.935...","POLYGON ((806676.934860112 6547474.821351372, ..."


In [9]:

# optionally to calculate the maximum intensity for each hazard point, this can be used, else a float can be used
max_intensity = []

for asset_id in f_assets.index:
    max_intensity.append(retrieve_max_intensity_by_asset(asset_id, overlay_assets, hazard_numpified_list))


In [11]:

# extract the assets that will be adapted
changed_assets = f_assets # testing with a few assets
# add new columns fragility_mod and haz_mod
changed_assets['fragility_mod'] = 1 #[0.3, 0.5, 0.8] #fraction [example considering no reduction] (1 = no reduction, 0 = invulnerable asset) DUMMY DATA FOR TESTING
# changed_assets['haz_mod'] = [np.max(x) for x in max_intensity] #meters [example adding wall of maximum flooding depth + 1 meter] (0 = no reduction in hazard intensity, 0.5 = 0.5 meter reduction in hazard intensity) DUMMY DATA FOR TESTING consider raising railway 0.5 meters
changed_assets['haz_mod'] = [np.max(x) if len(x) > 0 else 0 for x in max_intensity]

# TODO: automate infrastructure curve deduction from dictionary keys, now running with curve F8.1
hazard_intensity = infra_curves['F8.1'].index.values
fragility_values = (np.nan_to_num(infra_curves['F8.1'].values,nan=(np.nanmax(infra_curves['F8.1'].values)))).flatten()
maxdams_filt=max_damage_tables[max_damage_tables['ID number']=='F8.1']['Amount']

adaptation_run = run_damage_reduction_by_asset(geom_dict, overlay_assets, hazard_numpified_list, changed_assets, hazard_intensity, fragility_values, maxdams_filt)

#reporting
for asset_id, baseline_damages in adaptation_run[0].items():
    print(f'\nADAPTATION results for asset {asset_id}:')
    print(f'Baseline damages for asset {asset_id}: {baseline_damages[0]:.2f} to {baseline_damages[1]:.2f} EUR')
    print(f'Adapted damages for asset {asset_id}: {adaptation_run[1][asset_id][0]:.2f} to {adaptation_run[1][asset_id][1]:.2f} EUR')
    delta = tuple(adaptation_run[1][asset_id][i] - baseline_damages[i] for i in range(len(baseline_damages)))
    # percent_change = tuple((100 * (delta[i] / baseline_damages[i])) for i in range(len(baseline_damages)))
    percent_change = tuple((100 * (delta[i] / baseline_damages[i])) if baseline_damages[i] != 0 else 0 for i in range(len(baseline_damages)))
    print(f'Change (Adapted-Baseline): {delta[0]:.2f} to {delta[1]:.2f} EUR, {percent_change}% change, at a cost of {adaptation_run[2][asset_id]:.2f} EUR')

#TODO Check with economist: ammortization of adaptation cost over years of adaptation scenario
    #NEXT: adaptation_run returns (collect_inb_bl, collect_inb_adapt, adaptation_cost). These can be used to calculate the EAD for the adapted scenario (and damage reduction), and compare with the adaptation cost, which must be ammortized over the years of the adaptation scenario.




2024-07-29 16:27:22 - Calculating adapted damages for assets...


100%|██████████| 98/98 [01:32<00:00,  1.06it/s]

33 assets with no change.

ADAPTATION results for asset 12:
Baseline damages for asset 12: 543666.18 to 2458653.92 EUR
Adapted damages for asset 12: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -543666.18 to -2458653.92 EUR, (-100.0, -100.0)% change, at a cost of 22101136.68 EUR

ADAPTATION results for asset 270:
Baseline damages for asset 270: 0.00 to 112136.25 EUR
Adapted damages for asset 270: 0.00 to 0.00 EUR
Change (Adapted-Baseline): 0.00 to -112136.25 EUR, (0, -100.0)% change, at a cost of 652732.03 EUR

ADAPTATION results for asset 273:
Baseline damages for asset 273: 184843.07 to 184843.07 EUR
Adapted damages for asset 273: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -184843.07 to -184843.07 EUR, (-100.0, -100.0)% change, at a cost of 1661576.62 EUR

ADAPTATION results for asset 274:
Baseline damages for asset 274: 2757758.77 to 4467438.01 EUR
Adapted damages for asset 274: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -2757758.77 to -4467438.01 EUR, (-100.0, -100.0)% change, at 

In [ ]:
# EXPAND FOR VISUALISATION SCRIPT
# Create visualisation for the basin and the discharge points
from lonboard import Map, PolygonLayer, PathLayer, BaseLayer
import ast
# MIRACA color scheme
color_string = config.get('DEFAULT', 'miraca_colors')
miraca_colors = ast.literal_eval(color_string)

basin_id = 2080411370

    
# Set path for basin to add to visualization
basin_path = rf'C:\Data\Floods\basins\hybas_eu_{int(basin_id)}.shp'

# Generate the basin layer
basin = gpd.read_file(basin_path)
layer_basin = PolygonLayer.from_geopandas(basin,
    get_fill_color=miraca_colors['grey_200'],
    get_line_color=miraca_colors['primary blue'], get_line_width=70,
    auto_highlight=False,
    filled=True, opacity=0.2)

# Set path for the protected area to add to visualization
adapt_path = rf'C:\Data\interim\adaptations\test_haz_level_adapt.geojson'
adapt_area = gpd.read_file(adapt_path)
layer_protected_area = PolygonLayer.from_geopandas(adapt_area,
    get_fill_color=miraca_colors['green_success'],
    get_line_color=miraca_colors['green_400'], get_line_width=30,
    auto_highlight=False,
    filled=True, 
    opacity=0.2)

# Create layer for assets for visualization
layer_assets = PathLayer.from_geopandas(assets.drop(columns=['buffered']), get_width=2, get_color=miraca_colors['black'], auto_highlight=True, )
affected_assets = [asset_id for asset_id in list(set(overlay_assets.asset.values))]
layer_affected_assets = PathLayer.from_geopandas(assets.iloc[affected_assets].drop(columns=['buffered']), get_width=3, get_color=miraca_colors['red_danger'], auto_highlight=True)
layer_protected_assets = PathLayer.from_geopandas(f_assets.drop(columns=['buffered']), get_width=4, get_color=miraca_colors['green_success'], auto_highlight=True)
layers_assets = [layer_assets, layer_affected_assets, layer_protected_assets]
# Flood return period: H for frequent(RP10-20), M for 100 year return period (RP100) and L for extreme (RP2000)
return_period_str='L'
# Generate flood layers and protection layers for visualization
flood_plot_path=rf'Floods\Germany\basin_intersections\DERP_RW_{return_period_str}_4326_hybas_intersections\flood_DERP_RW_{return_period_str}_4326_{basin_id}.geojson'
flood_m = data_path / flood_plot_path
flood_gdf=gpd.read_file(flood_m)
layers_flood=[]
f_area_colors = {1:'blue', 3:'green'}
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            layers_flood.append(PolygonLayer.from_geopandas(subset_gdf, 
                                                            get_fill_color=miraca_colors[color_key], 
                                                            opacity=0.5, 
                                                            stroked=False))


layers=[]
if layer_basin is not None:
    layers.append(layer_basin)
else:
    print('No basin layer')

if layer_protected_area is not None:
    layers.append(layer_protected_area)
else:
    print('No protected area layer')

if layers_flood is not None:
    layers.extend(layers_flood)
else:
    print('No flood layers')

if layer_assets is not None:
    layers.extend(layers_assets)
else:
    print('No asset layer')
Voyager = 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'


# m=Map(layer_affected_assets, show_tooltip=True, basemap_style=Voyager)
m = Map(layers, show_tooltip=True, basemap_style=Voyager)



c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")
c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")
c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [11]:

m



Map(basemap_style='https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json', layers=[PolygonLayer(filled=…

In [1]:
assets.iloc[12]

NameError: name 'assets' is not defined